# **scDECA: Intergrating Gene-Cell interactions with Global Priors and Local Structures in Single-Cell Transcriptomic**

In [ ]:
import scDECA

In [ ]:
import scanpy as sc

adata = sc.read_h5ad('/data/project/bin000815/PriorGraph/scDECA/scDECA/Data/adata.h5ad')

In [ ]:
# --- scKNN 파라미터 설정 ---
scDECA.main.MAX_CELLS_BATCH_SIZE = 50000
scDECA.main.EXPRESSION_CUTOFF = 0.1
scDECA.main.NUM_LAYERS = 3
scDECA.main.DE_GENES_NUM = 3500
scDECA.main.EMBEDDING_DIM = 256

In [ ]:
scDECA.run_scDECA(
    adata,
    model_type='scgpt',  
    embedding_key='scGPT_gene_token', 
    pre_processing_flag=True, 
    human_flag=True,  
    number_of_batches=5,
    split_cells=False,
    max_epoch=250,
    model_name="scDECA_V1",
    device_str="cuda:0",
    num_heads=8,
    projection_dim=512
)

